In [1]:
import numpy as np
import cv2
import skimage
from skimage import segmentation, img_as_float
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def target_segmentation(image, max_iterations=500, lambda1=1, lambda2=1, smoothing=1):
    # Convert the image to a floating-point format
    image = img_as_float(image)
    
    # Perform level set evolution using the Chan-Vese algorithm
    ls = segmentation.morphological_chan_vese(
        image,
        num_iter=max_iterations,
        init_level_set="checkerboard",
        lambda1=lambda1,
        lambda2=lambda2,
        smoothing=smoothing
    )
    
    # ls is a boolean numpy array, where True represents the target region
    binary_target = ls.astype(np.uint8)
    
    # Crop the central region of the binary image
    return binary_target

def edge_extraction(binary_image, neighbor_threshold=10):
    # Define a 3x3 neighborhood kernel
    kernel = np.ones((3, 3), dtype=np.uint8)
    kernel[1, 1] = 0  # Exclude the center pixel
    
    # Count the number of target pixels in the neighborhood of each pixel
    neighbor_count = cv2.filter2D(binary_image, -1, kernel)
    
    # Create an edge matrix
    edges = np.zeros_like(binary_image)
    edges[(binary_image == 1) & (neighbor_count < neighbor_threshold)] = 1
    
    return edges

def azimuth_calculation(edges):
    # Find the coordinates of edge pixels
    coords = np.column_stack(np.where(edges > 0))
    
    # Check if there are enough points for calculation
    if len(coords) < 5:
        return None, None  # Not enough points for calculation
    
    # Calculate the minimum bounding rectangle using OpenCV
    rect = cv2.minAreaRect(coords.astype(np.float32))
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    
    # Extract the angle from the rectangle
    angle = rect[2]
    # OpenCV returns angles in the range [-90, 0)
    # Adjust the angle to match the azimuth definition
    if angle < -45:
        angle = 90 + angle
    else:
        angle = -angle
    
    azimuth_angle = angle % 360  # Ensure the angle is in the range [0, 360)
    
    return azimuth_angle, box

def line_detection_and_drawing(binary_target, image):
    # Find contours in the segmented image
    contours, _ = cv2.findContours(binary_target, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return image, None  # No contours found, return original image and angle as None

    # Select the largest contour
    contour = max(contours, key=cv2.contourArea)
    
    # Compute the minimum enclosing triangle
    area, trgl = cv2.minEnclosingTriangle(contour)
    trgl = trgl.astype(int)
    
    # Use the triangle vertices to define points for line drawing
    index_list = [0, 1, 2]
    index = np.argmax(trgl[:, :, 1])  # Find the lowest point (largest y-coordinate)
    index_list.remove(index)
    low_point = trgl[index]
    point_A = trgl[index_list[0]]
    point_B = trgl[index_list[1]]
    len_line_A = np.linalg.norm(point_A - low_point)
    len_line_B = np.linalg.norm(point_B - low_point)
    target_point = point_A if len_line_A > len_line_B else point_B

    # Draw the line on the image
    result_img = image.copy()
    cv2.line(result_img, tuple(low_point[0]), tuple(target_point[0]), (255, 0, 0), 1)
    
    # Calculate the angle of the line (0-180 degrees)
    delta_y = target_point[0][1] - low_point[0][1]
    delta_x = target_point[0][0] - low_point[0][0]
    angle = np.degrees(np.arctan2(delta_y, delta_x))
    if angle < 0:
        angle += 180  # Adjust the angle to the range [0, 180 degrees]
    
    return result_img, angle

# Read the image
image = cv2.imread('../_MSTAR/TRAIN/T72/HB03787.jpeg', cv2.IMREAD_GRAYSCALE)
# image = cv2.imread('../_MSTAR/TRAIN/2S1/HB19379.jpeg', cv2.IMREAD_GRAYSCALE)
# image = cv2.imread('../_MSTAR/TRAIN/BTR60/HB03817.jpeg', cv2.IMREAD_GRAYSCALE)
# image = cv2.imread('../_MSTAR/TRAIN/D7/HB20012.jpeg', cv2.IMREAD_GRAYSCALE)
# image = cv2.imread('../_MSTAR/TRAIN/ZSU234/HB19470.jpeg', cv2.IMREAD_GRAYSCALE)

# Step 1: Target segmentation
binary_target = target_segmentation(image)

# Step 2: Edge extraction
edges = edge_extraction(binary_target)

# Step 3: Azimuth calculation
azimuth_angle, bounding_box = azimuth_calculation(edges)

# Step 4: Line detection and angle calculation
result_with_line, line_angle = line_detection_and_drawing(binary_target, binary_target)

# Output the azimuth angle and line angle
print(f"Azimuth: {line_angle:.2f}°")

# Display the results
plt.figure(figsize=(12, 4))
plt.rcParams['figure.dpi'] = 200

plt.subplot(1, 4, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(binary_target, cmap='gray')
plt.title('Segmented Target')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(edges, cmap='gray')
plt.title('Extracted Edges')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(result_with_line, cmap='gray')
plt.title(f'Line Detection\nAngle: {line_angle:.2f}°')
plt.axis('off')

plt.tight_layout()
plt.show()

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


ValueError: Cannot convert from object to float64.